In [234]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapely
from shapely.geometry import Point
from geopandas.tools import sjoin
import json
import branca.colormap as cm
from jinja2 import Template

In [238]:
#Read Senate shapefile into GeoDataFrame
senate = gpd.read_file('senate2012/SENATE2012_POLY.shp')
senate["pred"] = 0
senate.set_value(3, 'pred', 1)
senate.set_value(10, 'pred', 1)
senate.set_value(8, 'pred', 1)
senate.set_value(20, 'pred', 1)
senate.set_value(35, 'pred', 1)

senate_predictions = senate[["SEN_DIST", "pred"]]
senate_dict = senate_predictions.set_index('SEN_DIST')['pred']

senate_colormap = cm.LinearColormap(
    ['#ccffcc', '#66ff66', '#009933'],
    vmin=senate_predictions.pred.min(), 
    vmax=senate_predictions.pred.max()
)

senate_colormap

/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated

In [246]:
m = folium.Map([42.2626, -71.8023], zoom_start=9)

color_dict = {key: senate_colormap(senate_dict[key]) for key in senate_dict.keys()}

g = folium.GeoJson(
    senate,
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['SEN_DIST']],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.9,
    },
    highlight_function = lambda feature: {
        'fillColor': '#ffaf00',
        'color': 'green', 
        'weight': 3
    }
).add_to(m)

g._template = Template("""
            {% macro script(this, kwargs) %}
               var {{this.get_name()}} = {}; 
               var popup_window = L.popup({maxWidth: '300'});   
               var district;

                {{this.get_name()}}.style = function(feature) {return feature.properties.style;};

                {{this.get_name()}}.highlightStyle = function(feature) {return {
                        weight: 5,
                        color: '#666',
                        dashArray: '',
                        fillOpacity: 0.7
                        };
                    };
                {{this.get_name()}}.onEachFeature = function onEachFeature(feature, layer) {
                    layer.on({
                        mouseover: function(e) {
                            e.target.setStyle({{this.get_name()}}.highlightStyle(e.target.feature));
                        },
                        click: function(e) {
                            district = feature.properties.SEN_DIST;
                            popup_window.setContent(district);  
                            e.target.bindPopup(popup_window);
                        },
                        mouseout: function(e) {
                            {{this.get_name()}}.geoJson.resetStyle(e.target);
                            }
                        });
                    };
                {{this.get_name()}}.geoJson = L.geoJson(
                    {% if this.embed %}{{this.style_data()}}{% else %}"{{this.data}}"{% endif %},{
                        style : {{this.get_name()}}.style,
                        onEachFeature: {{this.get_name()}}.onEachFeature
                        })
                    .addTo({{this._parent.get_name()}});
            {% endmacro %}
""")    

# And draw the map with additions, save
m.add_child(g)

senate_colormap.caption = 'Likelihood of Female Success'
m.add_child(senate_colormap)

m.save(os.path.join('results', 'Senate_GeoJSON_and_choropleth.html'))

In [243]:
#Read House shapefile into GeoDataFrame
house = gpd.read_file('house2012/HOUSE2012_POLY.shp')

house["pred"] = 0
house.set_value(3, 'pred', 1)
house.set_value(10, 'pred', 1)
house.set_value(8, 'pred', 1)
house.set_value(20, 'pred', 1)
house.set_value(35, 'pred', 1)

house_predictions = house[["REP_DIST", "pred"]]
house_dict = house_predictions.set_index('REP_DIST')['pred']

house_colormap = cm.LinearColormap(
    ['#ccffcc', '#66ff66', '#009933'],
    vmin=senate_predictions.pred.min(), 
    vmax=senate_predictions.pred.max()
)

house_colormap

/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/sarahferry/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future rele

In [247]:
m = folium.Map([42.2626, -71.8023], zoom_start=9)

color_dict = {key: house_colormap(house_dict[key]) for key in house_dict.keys()}
final_dict = house['REP_DIST']

g = folium.GeoJson(
    house,
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['REP_DIST']],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.9
    }
    
).add_to(m)

g._template = Template("""
            {% macro script(this, kwargs) %}
               var {{this.get_name()}} = {}; 
               var popup_window = L.popup({maxWidth: '300'});   
               var district;

                {{this.get_name()}}.style = function(feature) {return feature.properties.style;};

                {{this.get_name()}}.highlightStyle = function(feature) {return {
                        weight: 5,
                        color: '#666',
                        dashArray: '',
                        fillOpacity: 0.7
                        };
                    };
                {{this.get_name()}}.onEachFeature = function onEachFeature(feature, layer) {
                    layer.on({
                        mouseover: function(e) {
                            e.target.setStyle({{this.get_name()}}.highlightStyle(e.target.feature));
                        },
                        click: function(e) {
                            district = feature.properties.REP_DIST;
                            popup_window.setContent(district);  
                            e.target.bindPopup(popup_window);
                        },
                        mouseout: function(e) {
                            {{this.get_name()}}.geoJson.resetStyle(e.target);
                            }
                        });
                    };
                {{this.get_name()}}.geoJson = L.geoJson(
                    {% if this.embed %}{{this.style_data()}}{% else %}"{{this.data}}"{% endif %},{
                        style : {{this.get_name()}}.style,
                        onEachFeature: {{this.get_name()}}.onEachFeature
                        })
                    .addTo({{this._parent.get_name()}});
            {% endmacro %}
""")    

# And draw the map with additions, save
m.add_child(g)
house_colormap.caption = 'Likelihood of Female Success'
m.add_child(house_colormap)

m.save(os.path.join('results', 'House_GeoJSON_and_choropleth.html'))